# Guided Project: Building a Spam Filter with Naive Bayes

## Introduction

Our goal for this project is to create a spam filter for SMS messages wich has an accuracy over 80%.

To do that, we'll use the multinomial Naive Bayes algorithm along with a dataset of 5,572 SMS messages that were already classified by humans.

## The Data

The dataset we will be using was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). The data collection process is described in more details on this [page](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition), where we can also find some of the authors' papers.

## Inital Data Exploration

Let's start by reading in the dataset.

In [84]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

sms_spam = pd.read_csv('SMSSpamCollection', sep='\t', names=['Label', 'SMS'])

print(sms_spam.shape)
sms_spam.head()

(5572, 2)


,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Let's find out what percentage of the messages is spam and what percentage is ham ("ham" means non-spam).

In [85]:
sms_spam["Label"].value_counts(normalize=True)*100

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

About 87% of the messages are ham and the remaining 13% are spam.

## Splitting our dataset for training and testing

Once our spam filter is done, we'll need to test how good it is with classifying new messages. So before creating it, we split our dataset into two categories:
* A **training set**, which we'll use to "train" the computer how to classify messages.
* A **test set**, which we'll use to test how good the spam filter is with classifying new messages.

We're going to keep 80% of our dataset for training, and 20% for testing (we want to train the algorithm on as much data as possible, but we also want to have enough test data). The dataset has 5,572 messages, which means that:
* The training set will have 4,458 messages (about 80% of the dataset).
* The test set will have 1,114 messages (about 20% of the dataset).

For this project, our goal is to create a spam filter that classifies new messages with an accuracy greater than 80% — so we expect that more than 80% of the new messages will be classified correctly as spam or ham (non-spam).

Let's create a training and a test set! We start by randomizing the entire dataset.

In [86]:
randomized_data = sms_spam.sample(frac=1, random_state=1)

We then split the randomized dataset into a training and a test set.

In [87]:
# Calculate the index for split
training_test_index = round(len(randomized_data)*0.8)

# Training/Test split
training_set = randomized_data[:training_test_index].reset_index(drop=True)
print(training_set.shape)
test_set = randomized_data[training_test_index:].reset_index(drop=True)
print(test_set.shape)

(4458, 2)
(1114, 2)


Next, we calculate the percentages of spam and ham in each of these sets to see if they are representative of the full dataset.

In [88]:
training_set["Label"].value_counts(normalize=True)*100

ham     86.54105
spam    13.45895
Name: Label, dtype: float64

In [89]:
test_set["Label"].value_counts(normalize=True)*100

ham     86.804309
spam    13.195691
Name: Label, dtype: float64

The proportions of ham and spam are very similar for both the training and testing sets and also similar to the ones of the full dataset.

## Cleaning the Data

Before calculating all the probabilities required for the Naive Bayes algorithm, we first need to perform a bit of data cleaning. This will bring the data in a format that will allow us to extract easily all the information we need.

Essentially, we want to end up with a dataframe containing for each sms (each row) the counts of the number of times each word from the vocabulary (all unique words listed from all the sms) appear.

Let's print the head of the dataframe again and see what cleaning actions to take first.

In [90]:
# Before cleaning
training_set.head()

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


To be able to later generate the training_set vocabulary, we first need to clean the messages in the `SMS` column.

### Letter Case and Punctuation

What we are going to do first is:
* Remove all the punctuation form
* Transform every letter in every word to lower case

In [91]:
training_set["SMS"] = training_set["SMS"].str.replace('\W',' ')
training_set["SMS"] = training_set["SMS"].str.lower()

# After cleaning
training_set.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


### Creation of the Vocabulary

We can then create a vocabulary for the messages in the training set.

In [92]:
training_set["SMS"] = training_set["SMS"].str.split()

vocabulary = []

for index, row in training_set.iterrows():
    for word in row["SMS"]:
        if word not in vocabulary:
            vocabulary.append(word)
            
print(vocabulary)

['yep', 'by', 'the', 'pretty', 'sculpture', 'yes', 'princess', 'are', 'you', 'going', 'to', 'make', 'me', 'moan', 'welp', 'apparently', 'he', 'retired', 'havent', 'i', 'forgot', '2', 'ask', 'ü', 'all', 'smth', 'there', 's', 'a', 'card', 'on', 'da', 'present', 'lei', 'how', 'want', 'write', 'or', 'sign', 'it', 'ok', 'thk', 'got', 'then', 'u', 'wan', 'come', 'now', 'wat', 'kfc', 'its', 'tuesday', 'only', 'buy', 'meals', 'no', 'gravy', 'mark', 'dear', 'was', 'sleeping', 'p', 'pa', 'nothing', 'problem', 'ill', 'be', 'lt', 'gt', 'my', 'uncles', 'in', 'atlanta', 'wish', 'guys', 'great', 'semester', 'phone', 'which', 'your', 'another', 'number', 'greatest', 'test', 'of', 'courage', 'earth', 'is', 'bear', 'defeat', 'without', 'losing', 'heart', 'gn', 'tc', 'dai', 'what', 'this', 'can', 'send', 'resume', 'id', 'am', 'late', 'will', 'at', 'freemsg', 'why', 'haven', 't', 'replied', 'text', 'm', 'randy', 'sexy', 'female', 'and', 'live', 'local', 'luv', 'hear', 'from', 'netcollex', 'ltd', '08700621

We were able to successfully generate the vocabulary list for our training set. This list contains 7783 elements, meaning that messages in the training set are made from a list of 7783 unique words.

### The Finale Training Set

Next, we use this vocabulary we created to make the data transformation. Eventually, we're going to create a new DataFrame. However, we'll first build a dictionary that we'll then convert to the DataFrame we need.

In the block of code below, we:
* start by initializing a dictionary named `word_counts_per_sms`, where each key is a unique word (a string) from the vocabulary, and each value is a list of the length of training set, where each element in the list is a 0.
* loop over `training_set['SMS']` using at the same time the `enumerate()` function to get both the index and the SMS message (index and sms).
    * using a nested loop, we loop over `sms` (where `sms` is a list of strings, where each string represents a word in a message).
        * we increment `word_counts_per_sms[word][index]` by 1. 

In [93]:
word_counts_per_sms = {unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(training_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1
        
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,yep,by,the,pretty,sculpture,yes,princess,are,you,going,...,beauty,hides,secrets,n8,jewelry,related,trade,arul,bx526,wherre
0,1,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


The `word_counts` dataframe was successfully created. However, it misses the full messages and, more importantly, the labels.

In the following block of code, we add those two columns to our `word_counts` dataframe. We rename the resulting dataframe `training_set_clean`.

In [94]:
training_set_clean = pd.concat([training_set["Label"], training_set["SMS"], word_counts], axis=1)
training_set_clean.head()

,Label,SMS,yep,by,the,pretty,sculpture,yes,princess,are,...,beauty,hides,secrets,n8,jewelry,related,trade,arul,bx526,wherre
0,ham,"[yep, by, the, pretty, sculpture]",1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Now that we're done with data cleaning and have a training set to work with, we can begin creating the spam filter.

## Building the Spam Filter

### Calculation of constants

Recall that our spam filter will be based on a multinomial Naive Bayes algorithm. This algorithm needs to know the probability values of the two following equations to be able to classify new messages:

$$P(Spam|w_{1}, w_{2},..., w_{n}) \propto  P(Spam) \cdot \prod_{i = 1}^{n} P(w_{i}|Spam)$$

$$P(Ham|w_{1}, w_{2},..., w_{n}) \propto  P(Ham) \cdot \prod_{i = 1}^{n} P(w_{i}|Ham)$$

Also, to calculate P(w<sub>i</sub>|Spam) and P(w<sub>i</sub>|Ham) inside the formulas above, recall that we need to use these equations:

$$P(w_{i}|Spam) = \frac{N_{w_{i}|Spam}+\alpha }{N_{Spam}+\alpha \cdot N_{Vocabulary}}$$

$$P(w_{i}|Ham) = \frac{N_{w_{i}|Ham}+\alpha }{N_{Ham}+\alpha \cdot N_{Vocabulary}}$$

Some of the terms in the four equations above will have the same value for every new message. As a start, let's first calculate:
* P(Spam) and P(Ham)
* N<sub>Spam</sub>, N<sub>Ham</sub>, N<sub>Vocabulary</sub>

N<sub>Spam</sub> = number of words in all the spam messages

N<sub>Ham</sub> = number of words in all the non-spam messages

We'll also use Laplace smoothing and set $\alpha$ = 1.

In the next blocks of code, and for the training set only, we:
* calculate P(Spam) and P(Ham)
* calculate N<sub>Spam</sub>, N<sub>Ham</sub> and N<sub>Vocabulary</sub>
* initiate a variable named `alpha` with a value of 1

In [95]:
# Get all spam messages from the training set
spams = training_set_clean[training_set_clean["Label"] == 'spam']

# Get all ham messages from the training set
hams = training_set_clean[training_set_clean["Label"] == 'ham']

p_spam = len(spams) / len(training_set_clean)
p_ham = len(hams) / len(training_set_clean)

print(p_spam)
print(p_ham)

0.13458950201884254
0.8654104979811574


In [96]:
# Total number of unique words in all messages of the training set
n_vocabulary = len(vocabulary)

# Number of words in all the spam messages
n_spam = spams.iloc[:,2:].sum().sum().astype(int)

# Number of words in all the ham messages
n_ham = hams.iloc[:,2:].sum().sum().astype(int)

print(n_vocabulary)
print(n_spam)
print(n_ham)

7783
15190
57237


In [97]:
# Initialization of the variable alpha for Laplace smoothing
alpha = 1

### Calculation of Parameters

All the terms we calculated thus far (`p_spam`, `p_ham`, `n_vocabulary`, `n_spam`, `n_ham`) will have constant values in our equations for every new message (regardless of the message or each individual word in the message).

However, P(wi|Spam) and P(wi|Ham) will vary depending on the individual words. For instance, P("secret"|Spam) will have a certain probability value, while P("cousin"|Spam) or P("lovely"|Spam) will most likely have other values.

Although both P(wi|Spam) and P(wi|Ham) vary depending on the word, the probability for each individual word is constant for every new message.

For instance, let's say we receive two new messages:

* "secret code"
* "secret party 2night"

We'll need to calculate P("secret"|Spam) for both these messages, and we can use the training set to get the values we need to find a result for the equation below:

$$P("secret"|Spam) = \frac{N_{"secret"|Spam}+\alpha }{N_{Spam}+\alpha \cdot N_{Vocabulary}}$$

The steps we take to calculate P("secret"|Spam) will be identical for both of our new messages above, or for any other new message that contains the word "secret". The key detail here is that calculating P("secret"|Spam) only depends on the training set, and as long as we don't make changes to the training set, P("secret"|Spam) stays constant. The same reasoning also applies to P("secret"|Ham).

This means that we can use our training set to calculate the probability for each word in our vocabulary. If our vocabulary contained only the words "lost", "navigate", and "sea", then we'd need to calculate six probabilities:

* P("lost"|Spam) and P("lost"|Ham)
* P("navigate"|Spam) and P("navigate"|Ham)
* P("sea"|Spam) and P("sea"|Ham)

We have 7,783 words in our vocabulary, which means we'll need to calculate a total of 15,566 probabilities. For each word, we need to calculate both P(wi|Spam) and P(wi|Ham).

In more technical language, the probability values that P(wi|Spam) and P(wi|Ham) will take are called **parameters**.

The fact that we calculate so many values before even beginning the classification of new messages makes the Naive Bayes algorithm very fast (especially compared to other algorithms). When a new message comes in, most of the needed computations are already done, which enables the algorithm to almost instantly classify the new message.

If we didn't calculate all these values beforehand, then all these calculations would need to be done every time a new message comes in.

Let's now calculate all the parameters using the equations below:

$$P(w_{i}|Spam) = \frac{N_{w_{i}|Spam}+\alpha }{N_{Spam}+\alpha \cdot N_{Vocabulary}}$$

$$P(w_{i}|Ham) = \frac{N_{w_{i}|Ham}+\alpha }{N_{Ham}+\alpha \cdot N_{Vocabulary}}$$

We start by initializing two dictionaries, where each key-value pair is a unique word (from our vocabulary) represented as a string, and the value is 0. We'll need one dictionary to store the parameters for P(wi|Spam) and another for the parameters for P(wi|Ham).

In [98]:
spam_params = {unique_word: 0 for unique_word in vocabulary}
ham_params = {unique_word: 0 for unique_word in vocabulary}

We then iterate over the vocabulary and, for each word, calculate P(w<sub>i</sub>|Spam) and P(w<sub>i</sub>|Ham) using the formulas we mentioned above.

In [99]:
for word in vocabulary:
    n_word_spam = spams[word].sum() 
    n_word_ham = hams[word].sum()
    p_word_given_spam = (n_word_spam + alpha) / (n_spam + alpha * n_vocabulary) 
    p_word_given_ham = (n_word_ham + alpha) / (n_ham + alpha * n_vocabulary) 
    spam_params[word] = p_word_given_spam
    ham_params[word] = p_word_given_ham

Now that we have calculated all the constants and parameters we need, we can start creating the spam filter.

### Creation of the Filter

The spam filter can be understood as a function that:

* Takes in as input a new message (w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>)
* Calculates P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) and P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>)
* Compares the values of P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) and P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), and:
    * If P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) > P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), then the message is classified as ham.
    * If P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) < P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), then the message is classified as spam.
    * If P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) = P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), then the algorithm may request human help.
    
Some new messages may contain words that are not part of the vocabulary. We will simply ignore these words when we're calculating the probabilities.

Now we'll write the code for calculating `p_spam_given_message` and `p_ham_given_message`, and then we'll use the function to classify two new messages. After that, we'll classify all the 1,114 messages in our test set.

In [100]:
import re

def classify(message):

    # Cleaning of the new message
    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    # Initialization of the variables
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in spam_params:
            p_spam_given_message *= spam_params[word]
        if word in ham_params:
            p_ham_given_message *= ham_params[word]
        else:
            pass
        
    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

We now test our `classify()` function on two new messages, one obviously spam and another one obviously ham. These two messages are the following:
* 'WINNER!! This is the secret code to unlock the money: C3421.'
* 'Sounds good, Tom, then see u there"

In [101]:
test_message_1 = 'WINNER!! This is the secret code to unlock the money: C3421.'
test_message_2 = 'Sounds good, Tom, then see u there'

print(classify(test_message_1))
print(classify(test_message_2))

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam
None
P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham
None


Our spam filter correctly classified the two messages. Let's now move to the proper validation phase!

## Assessing the Filter's accuracy

We'll now try to determine how well the spam filter does on our test set of 1,114 messages.

The algorithm will output a classification label for every message in our test set, which we'll be able to compare with the actual label.

First off, we'll change the `classify()` function that we wrote previously to return the labels instead of printing them. 

In [102]:
def classify_test_set(message):

    # Cleaning of the new message
    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    # Initialization of the variables
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in spam_params:
            p_spam_given_message *= spam_params[word]
        if word in ham_params:
            p_ham_given_message *= ham_params[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'needs human classification'

Now that we have a function that returns labels instead of printing them, we can use it to create a new column in our test set.

In [103]:
test_set['predicted'] = test_set['SMS'].apply(classify_test_set)
test_set.head(10)

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham
5,ham,But my family not responding for anything. Now...,ham
6,ham,U too...,ham
7,ham,Boo what time u get out? U were supposed to ta...,ham
8,ham,Genius what's up. How your brother. Pls send h...,ham
9,ham,I liked the new mobile,ham


Looking at the head of `test_set`, it seems our filter did pretty well at classifying the messages.

We can compare the predicted values with the actual labels for all 1,114 messages to measure how good our spam filter is with classifying new messages. To make the measurement, we'll use **accuracy** as a metric:

$$Accuracy = \frac{number\:of\:correctly\:classified\:messages}{total\:number\:of\:classified\:messages}$$

In [104]:
correct = 0
total = len(test_set)

for index, row in test_set.iterrows():
    if row["Label"] == row["predicted"]:
        correct += 1
        
accuracy = correct / total
print(round(accuracy,4))

0.9874


Our spam filter has an accuracy of 98.74% on the test set. This is an excellent result.

## Conclusion

In this project, we managed to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. The filter had an accuracy of 98.74% on the test set, which is an excellent result. We initially aimed for an accuracy of over 80%, but we managed to do way better than that.

Potential next steps for this project would be:
* isolate the 14 messages that were classified incorrectly and try to figure out why the algorithm reached the wrong conclusions.
* Make the filtering process more complex by making the algorithm sensitive to letter case.